In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
text = """
در مسئلههای زي ستی روشهای متفاوت خطی مانند فاصله اقلیدس ی برای محاسبه فاصله بردارها استفاده میشود. اخیرا روشهای 
جديدی در شبکه های عصبی به نام شبکههای دوقلو معرفی شده كه میتوانند از يک شبکه با وزن مشترک برای مقايسه دو بردار 
استفاده نمايند. اگر دو ورودی متعلق به يک گونه باشند، ابعاد بردارها را در فضا به صورتی تغییر میدهد كه اختالف آن ها كم و
نزديک به صفر و در غیر اين صورت اختالف دو ورودی نزديک به يک شود. 
"""

In [7]:
!pip install googletrans==4.0.0-rc1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 3.1 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 1.4 MB 42.0 MB/s 
     |████████████████████████████████| 65 kB 4.0 MB/s 
     |████████████████████████████████| 53 kB 2.4 MB/s 
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17416 sha256=adc4ae53cd5dba9c1b5301197213c64e1cb5d5ecf2eb41954d581c7a603416fd
  Stored in directory: /root/.cache/pip/wheels/43/34/00/4fe71786ea6d12314b29037620c36d857e5d104ac2748bf82a
Successfully built googletrans


In [8]:
import googletrans
from googletrans import Translator

translator = Translator()
en = translator.translate(text,dest='en')
en_text = en.text

In [9]:
sentences = sent_tokenize(en_text)

In [ ]:
# !pip install transformers sentencepiece

In [10]:
!pip install sentence-splitter
!pip install transformers
!pip install SentencePiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 44 kB 2.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 15.2 MB/s 
     |████████████████████████████████| 101 kB 4.3 MB/s 
     |████████████████████████████████| 6.6 MB 10.1 MB/s 
     |████████████████████████████████| 596 kB 10.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 15.0 MB/s 


In [11]:
#importing the PEGASUS Transformer model
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
 
model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

Downloading:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

In [12]:
#setting up the model
def get_response(input_text,num_return_sequences = 1):
  batch = tokenizer.prepare_seq2seq_batch([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=60,num_beams=10, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

In [13]:
paraprased = []
for s in sentences:
  paraprased.append(get_response(s))

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3557: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


In [ ]:
# def get_paraphrased_sentences(model, tokenizer, sentence, num_return_sequences=5, num_beams=5):
#   # tokenize the text to be form of a list of token IDs
#   inputs = tokenizer([sentence], truncation=True, padding="longest", return_tensors="pt")
#   # generate the paraphrased sentences
#   outputs = model.generate(
#     **inputs,
#     num_beams=num_beams,
#     num_return_sequences=num_return_sequences,
#   )
#   # decode the generated sentences using the tokenizer to get them back to text
#   return tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [ ]:
from nltk.tokenize import word_tokenize

In [ ]:
# paraphrased=[]
# for s in ps:
#   if len(word_tokenize(s))>= 3:
#     tmp = get_paraphrased_sentences(model, tokenizer, s, num_beams=10, num_return_sequences=1)
#     paraphrased.append(tmp)
#   else:
#     paraphrased.append([s])

In [15]:
joined_list=[]
for p in paraprased:
  joined_list.append(p[0])

In [16]:
joined_text = "\n".join(joined_list)

In [17]:
persian_text = translator.translate(joined_text,dest='fa')